In [1]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import gradio as gr
from PIL import Image
from PIL import ImageDraw

In [2]:


# Create an Image Analysis client
client = ImageAnalysisClient(
    endpoint='https://westus2.api.cognitive.microsoft.com/',
    credential=AzureKeyCredential('07fdb657daed4e34af2541c636ee1a75')
)

# Get a caption for the image. This will be a synchronously (blocking) call.
result = client.analyze_from_url(
    image_url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
    visual_features=[VisualFeatures.OBJECTS],
    gender_neutral_caption=True,  # Optional (default is False)
)



In [7]:
subscription_key = "7c9ff32669cb48b29e076df67c8d60c7"
endpoint = "https://b03-computevision.cognitiveservices.azure.com/"


def request_vision(image_path, features):
    api_url = f"{endpoint}/computervision/imageanalysis:analyze"
    params = {
        "features": features,
        "model-version": "latest",
        "language": "en",
        "api-version": "2024-02-01"
    }
    # 요청 헤더 설정
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Content-Type": "application/octet-stream"
    }

    # 요청 데이터 설정
    data = {
        "url": "https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png"
    }
    
    with open(image_path,'rb') as image:
        image_data = image.read()


    response = requests.post(api_url, headers=headers, params=params, data=image_data)
    print(response.json())
    # 요청 보내기

    return response.json()

In [24]:
def change_image(image_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    response_json=request_vision(image_path, 'objects')
    print(response_json)
    
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red") 
    elif 'tagsResult' in response_json and "values" in response_json["tagsResult"]:
        for obj in response_json['tagsResult']['values']:
            print(obj['name'],int(obj['confidence']*100))
        
    return image

In [25]:
import gradio as gr

with gr.Blocks() as demo:
    input_image = gr.Image(label='이미지 선택',type='filepath')
    output_image = gr.Image(label='출력 이미지',type='pil',interactive=False)
    
    input_image.change(fn=change_image,inputs=[input_image],outputs=[output_image])
    
demo.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 816, 'height': 612}, 'objectsResult': {'values': [{'boundingBox': {'x': 141, 'y': 206, 'w': 613, 'h': 231}, 'tags': [{'name': 'tool', 'confidence': 0.718}]}]}}
{'modelVersion': '2023-10-01', 'metadata': {'width': 816, 'height': 612}, 'objectsResult': {'values': [{'boundingBox': {'x': 141, 'y': 206, 'w': 613, 'h': 231}, 'tags': [{'name': 'tool', 'confidence': 0.718}]}]}}
141 206 613 231


In [ ]:

print("Image analysis results:")
# Print caption results to the console
print(" Caption:")
if result.caption is not None:
    print(f"   '{result.caption.text}', Confidence {result.caption.confidence:.4f}")

# Print text (OCR) analysis results to the console
print(" Read:")
if result.read is not None:
    for line in result.read.blocks[0].lines:
        print(f"   Line: '{line.text}', Bounding box {line.bounding_polygon}")
        for word in line.words:
            print(f"     Word: '{word.text}', Bounding polygon {word.bounding_polygon}, Confidence {word.confidence:.4f}")